In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
from datetime import date
from tqdm.notebook import tqdm

In [40]:
def get_film_links(first_page, last_page):

    
    df = pd.DataFrame(columns=['film_full_name','year', 'counry','film_url'])

    current_date = str.replace(str(date.today()), '-', '_')
    current_time = time.strftime('%H_%M_%S', time.localtime())
    temporal_txt_path = 'fil_scraper_temp_file_' + \
        current_date + '_' + current_time + '.csv'

    for page in tqdm(range(first_page,last_page+1)):

        # search_url = f'https://www.kinopoisk.ru/top/navigator/m_act[egenre]/22%2C12%2C27%2C23%2C1747%2C9%2C25%2C999%2C24%2C26%2C18%2C1751/m_act[country]/2%2C13%2C2%2C13/m_act[years]/2000%3A/m_act[rating]/5%3A/m_act[country_or]/on/m_act[is_film]/on/order/rating/page/{page}/#results'
        # req = requests.get(search_url)
        path_to_html = f'./film_search/{str(page)}.html'

        with open(path_to_html) as fp:
            soup = BeautifulSoup(fp, 'html.parser')

        mydivs = soup.find_all("div", {"class": "info"})

        for div in mydivs:

            film_url = div.a['href']
            film_full_name = div.a.text
            film_year = re.search(r'\(\d*\)',div.span.text)[0]

            text = div.find("span", {"class": "gray_text"}).text
            country = re.search(r'\w*',text.split()[0])[0]
            genres_dirt = re.search(r'\(.*\)',text)[0]
            genres = re.findall(r'\w+',genres_dirt)

            print(film_url, film_full_name, country, genres)

            with open(temporal_txt_path, 'a') as temp_file:
                temp_file.write(
                    ';'.join([str(film_full_name), str(film_year), str(country), str(film_url)])+'\n')

            new_row = pd.DataFrame({'film_full_name':film_full_name,'year':film_year,'counry':country, 'film_url':film_url},index=[film_full_name])
            df = pd.concat([df.loc[:,:],new_row],)

        # time.sleep(10)


    df = df.reset_index(drop=True)
    df.to_csv('fil_scraper.csv', sep=';')
    
    return df

In [41]:
df = get_film_links(1,11)

  0%|          | 0/11 [00:00<?, ?it/s]

https://www.kinopoisk.ru/level/1/film/4791604/ Смешарики. Зимние серии Россия ['мультфильм', 'детский', 'семейный']
https://www.kinopoisk.ru/level/1/film/4791590/ Смешарики. Новогодние серии Россия ['мультфильм', 'детский', 'семейный']
https://www.kinopoisk.ru/level/1/film/4299813/ Байкал. Удивительные приключения Юмы Россия ['семейный']
https://www.kinopoisk.ru/level/1/film/840526/ Евгений Гришковец: Прощание с бумагой  (ТВ) Россия ['драма', 'комедия']
https://www.kinopoisk.ru/level/1/film/1009577/ Евгений Гришковец: Шепот сердца Россия ['драма']
https://www.kinopoisk.ru/level/1/film/258321/ Евгений Гришковец: Как я съел собаку  (видео) Россия ['драма', 'мелодрама', 'комедия']
https://www.kinopoisk.ru/level/1/film/5237750/ Не хороните меня без Ивана Россия ['драма']
https://www.kinopoisk.ru/level/1/film/1178267/ Сестрёнка Россия ['драма', 'военный']
https://www.kinopoisk.ru/level/1/film/471493/ Юнона и Авось  (ТВ) Россия ['драма', 'музыка']
https://www.kinopoisk.ru/level/1/film/139263